T VS F CLASSIFICATION

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

binary_data_path = '/content/drive/MyDrive/NLP_Workspace/Data/BERT_data/'

df = pd.read_csv(binary_data_path + "F_vs_T_data_withEmoji.csv")
df = df.dropna(how='any',axis=0)
df.sample(10)

,label,text
191116,1,agree completely lived new york used public tr...
96003,0,hahahahahahahahaha love
150142,1,see gender anything intj guy emotional bubbly ...
176175,1,highlight wall text right click choose less wo...
114950,1,hmm turns test w sexual variant self-image get...
98032,0,haha neither seems far stressful <emoji>
97672,0,remember thread commented shale st grader desc...
186487,1,lol title threads website amuse <emoji> contra...
25890,0,'re saying completely true think judge cogniti...
166024,1,bd above- version sagan-smile love man thanks ...


In [3]:
DOWNSAMPLE = 30000
df_balanced = df.sample(DOWNSAMPLE)
df_balanced['label'].value_counts()

1    15025
0    14975
Name: label, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_balanced, test_size=0.15)

In [5]:
train.head(3)

,label,text
9010,0,scottish welsh norman french hessian mother si...
31002,0,apparently esfj guys many people something lik...
59906,0,got istj friend help time time younger views s...


In [6]:
test.head(3)

,label,text
187064,1,imo academic integrity requires phd coursework...
125548,1,unhealthy estj ex estj
187169,1,due type job position play within family setti...


In [7]:
ourpur_dir = binary_data_path + "TvsF_BERT_withEmojis/"
train.to_csv(ourpur_dir+"T_vs_F_train_data.csv", index=False)
test.to_csv(ourpur_dir+"T_vs_F_test_data.csv", index=False)

In [8]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.5 MB/s 
     |████████████████████████████████| 212 kB 72.9 MB/s 
     |████████████████████████████████| 132 kB 65.9 MB/s 
     |████████████████████████████████| 182 kB 74.4 MB/s 
     |████████████████████████████████| 127 kB 65.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [9]:
data_files = {"train": ourpur_dir+"T_vs_F_train_data.csv", "test": ourpur_dir+"T_vs_F_test_data.csv"}

from datasets import load_dataset
squad_it_dataset = load_dataset("csv", data_files=data_files, sep=',')



Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c20d27ca1ab14663/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 25500
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 4500
    })
})

In [11]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.5 MB/s 
     |████████████████████████████████| 7.6 MB 60.2 MB/s 


In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [14]:
TRAIN_NUM = int(DOWNSAMPLE * 0.85)
TEST_NUM = int(DOWNSAMPLE * 0.15)
TEST_NUM

4500

In [15]:
train_dataset = squad_it_dataset["train"].shuffle(seed=42).select([i for i in list(range(TRAIN_NUM))])
test_dataset = squad_it_dataset["test"].shuffle(seed=42).select([i for i in list(range(TEST_NUM))])

tokenized_train = train_dataset.map(preprocess_function)
tokenized_test = test_dataset.map(preprocess_function)

  0%|          | 0/25500 [00:00<?, ?ex/s]

  0%|          | 0/4500 [00:00<?, ?ex/s]

In [16]:
tokenized_train[0]

{'label': 1,
 'text': 'estp recently become pretty close infp hang frequently etc calls best friend reluctant say anyways pissed yesterday',
 'input_ids': [101,
  9765,
  2361,
  3728,
  2468,
  3492,
  2485,
  1999,
  22540,
  6865,
  4703,
  4385,
  4455,
  2190,
  2767,
  11542,
  2360,
  4312,
  2015,
  9421,
  7483,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [17]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [19]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [20]:
training_args = TrainingArguments(
    
    output_dir= ourpur_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [21]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25500
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7970
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Step,Training Loss
500,0.677800
1000,0.664200
1500,0.654300
2000,0.618200
2500,0.601000
3000,0.600900
3500,0.530800
4000,0.483300
4500,0.467500
5000,0.426700


Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT_withEmojis/checkpoint-500
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT_withEmojis/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT_withEmojis/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT_withEmojis/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT_withEmojis/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT_withEmojis/checkpoint-1000
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT_withEmojis/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT_withEmojis/checkpoint-1000

TrainOutput(global_step=7970, training_loss=0.47636318805077144, metrics={'train_runtime': 598.0465, 'train_samples_per_second': 213.194, 'train_steps_per_second': 13.327, 'total_flos': 990962736806736.0, 'train_loss': 0.47636318805077144, 'epoch': 5.0})

In [22]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 16


<ipython-input-19-b1430cee0b64>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.9902300238609314,
 'eval_accuracy': 0.6286666666666667,
 'eval_f1': 0.6203135650988412,
 'eval_runtime': 5.7824,
 'eval_samples_per_second': 778.219,
 'eval_steps_per_second': 48.768,
 'epoch': 5.0}